In [18]:
%load_ext autoreload
%autoreload 2
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
print("import torch")
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

print("import lightning")
import lightning as pl
from lightning.pytorch.callbacks import LearningRateMonitor, Callback
from lightning.pytorch import Trainer, LightningModule

print("import transformers")
from transformers.models.gpt2 import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Config

print("import datasets")
from datasets import load_dataset

from collections import OrderedDict

print("import model")
import model
import knn
print("Done")

import torch
import lightning
import transformers
import datasets
import model
Done


In [5]:
teacher_name = "gpt2-large"

print("Loading teacher")

teacher = GPT2LMHeadModel.from_pretrained(teacher_name)
tokenizer = GPT2Tokenizer.from_pretrained(teacher_name)
config = teacher.config
print("Done")z

Loading teacher


Done


In [37]:
print("kNN memory")
memory = knn.KNNMemory("orin", config.n_embd, config.n_embd)
print("Done")

Global seed set to 42
Found cached dataset ag_news (/home/consciouscode/.cache/huggingface/datasets/ag_news/all/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/consciouscode/.cache/huggingface/datasets/ag_news/all/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-7ad8216966f98598.arrow


Done


In [38]:
print("Begin training")

# Train the model
model = KnowledgeDistillationModel(teacher, student, memory, tokenizer)
trainer = pl.Trainer(accelerator="gpu", max_epochs=3, callbacks = z)
trainer.fit(model, dataloader)

# Save the distilled model
student.save_pretrained("distilled_gpt2")
torch.set_float32_matmul_precision('medium')
trainer = pl.Trainer(
    accelerator="gpu", max_epochs=3,
    callbacks=[
        LearningRateMonitor(logging_interval='step'),
        FrequentCheckpoint(save_steps=1000, output_dir="checkpoints")
    ]
)
trainer.fit(
    KnowledgeDistillationModule(teacher, model.InfoDistilleryLMHeadModel(student), memory),
    datamodule=WikiText103DataModule(dataset)
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                    | Params
----------------------------------------------------------
0 | teacher_model | GPT2LMHeadModel         | 124 M 
1 | student_model | InfoDistilleryGPT2Model | 66.9 M
2 | distill_loss  | KLDivLoss               | 0     
3 | ce_loss       | CrossEntropyLoss        | 0     
----------------------------------------------------------
191 M     Trainable params
384       Non-trainable params
191 M     Total params
765.476   Total estimated model params size (MB)


Begin training


Training: 0it [00:00, ?it/s]

input_ids [tensor([22401,  9914, 44142, 31206, 44142,  1273, 26788, 42268],
       device='cuda:0'), tensor([  520,   306,   290, 11023,  4536,  3320, 34068,  2431],
       device='cuda:0'), tensor([   13,   293, 18493,   912, 48701,  5268, 21710,   905],
       device='cuda:0'), tensor([15682, 29403, 10130, 11474,   284,  3205,   287, 17873],
       device='cuda:0'), tensor([30358, 24324,   520,  1475,   477,    11, 26603,   625],
       device='cuda:0'), tensor([ 5157,   446,  3320,  3742,    12,   887,  6119, 10610],
       device='cuda:0'), tensor([20008, 22724,     6,   422,  2435, 20173,   357,   357],
       device='cuda:0'), tensor([  262, 43226, 30096,  8774,  1700,  6280,  2969,  2937],
       device='cuda:0'), tensor([2619,  357,  357, 8050,   11,  406,    8, 1404], device='cuda:0'), tensor([  357, 12637, 12637, 37709, 24380,  1666,  3486,  3727],
       device='cuda:0'), tensor([12637,     8,     8,   357,   649,   357,   532,  4792],
       device='cuda:0'), tensor([    8,

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │   LearningRateMonitor(logging_interval='step'),                                           │
│    7 │   FrequentCheckpoint(save_steps=1000, output_dir="checkpoints")                           │
│    8 ])                                                                                          │
│ ❱  9 trainer.fit(model, dataloader)                                                              │
│   10                                                                                             │
│                                                                                                  │
│ /home/consciouscode/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/trainer.py:520 │
│ in fit                                                                                           │
│                                                                                                  │
│    517 │   │   """                                                                               │
│    518 │   │   model = _maybe_unwrap_optimized(model)                                            │
│    519 │   │   self.strategy._lightning_module = model                                           │
│ ❱  520 │   │   call._call_and_handle_interrupt(                                                  │
│    521 │   │   │   self, self._fit_impl, model, train_dataloaders, val_dataloaders, datamodule,  │
│    522 │   │   )                                                                                 │
│    523                                                                                           │
│                                                                                                  │
│ /home/consciouscode/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:44 in  │
│ _call_and_handle_interrupt                                                                       │
│                                                                                                  │
│    41 │   │   if trainer.strategy.launcher is not None:                                          │
│    42 │   │   │   return trainer.strategy.launcher.launch(trainer_fn, *args, trainer=trainer,    │
│    43 │   │   else:                                                                              │
│ ❱  44 │   │   │   return trainer_fn(*args, **kwargs)                                             │
│    45 │                                                                                          │
│    46 │   except _TunerExitException:                                                            │
│    47 │   │   _call_teardown_hook(trainer)                                                       │
│                                                                                                  │
│ /home/consciouscode/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/trainer.py:559 │
│ in _fit_impl                                                                                     │
│                                                                                                  │
│    556 │   │   │   model_provided=True,                                                          │
│    557 │   │   │   model_connected=self.lightning_module is not None,                            │
│    558 │   │   )                                                                                 │
│ ❱  559 │   │   self._run(model, ckpt_path=ckpt_path)                                             │
│    560 │   │                                                                                     │
│    561 │   │   assert self.state.stopped                   